In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import copy
import os
import matplotlib.pyplot as plt
import matplotlib
from matplotlib_inline.backend_inline import set_matplotlib_formats
import seaborn as sns
import torch
import torch.distributions as dist
from signatureshape.animation import fetch_animation_id_set, fetch_animations
from signatureshape.animation.src.mayavi_animate import mayavi_animate

import extratorch as etorch
import shapeflow as sf

In [ ]:
# make reproducible
seed = torch.manual_seed(0)

# better plotting
set_matplotlib_formats("pdf", "svg")
matplotlib.rcParams.update({"font.size": 12})
set_matplotlib_formats("pdf", "svg")
plt.style.use("tableau-colorblind10")
sns.set_style("white")

In [ ]:
# fetch data as eulers
# we assume all have the same skeleton
print("Loading mocap data:")
# walk  data
walk_subjects = ["07", "08", "35", "16"]
walk_animations = []
for s in walk_subjects:
    for t in fetch_animations(100, subject_file_name=(s + ".asf")):
        if t[2][:4] == "walk":
            walk_animations.append(t[1])

# run data
run_subjects = ["09", "16", "35"]
run_animations = []
run_skeletons = []
for s in run_subjects:
    for t in fetch_animations(100, subject_file_name=(s + ".asf")):
        if t[2][:3] == "run":
            run_skeletons.append(t[0])
            run_animations.append(t[1])

print("Convert to array:")
walk_angle_array = sf.utils.animation_to_eulers(
    walk_animations,
    reduce_shape=False,
    remove_root=True,
    deg2rad=True,
    max_frame_count=240,
)
run_angle_array = sf.utils.animation_to_eulers(
    run_animations,
    reduce_shape=False,
    remove_root=True,
    deg2rad=True,
    max_frame_count=240,
)

In [ ]:
# tensors on the form
# (motion, time, joint)
walk_angle_tensor = torch.tensor(walk_angle_array, dtype=torch.float32)
run_angle_tensor = torch.tensor(run_angle_array, dtype=torch.float32)

pre_shape_walk = walk_angle_tensor.shape
pre_shape_run = run_angle_tensor.shape
num_frames = min(pre_shape_walk[1], pre_shape_run[1])

nonzero = torch.argwhere(
    torch.sum(torch.abs(torch.diff(walk_angle_tensor, dim=1)), dim=[0, 1]) > 0.0
).flatten()

In [ ]:
# insert best shapes here
skip_frames = 4

# cut and reduce frames
walk_angles = walk_angle_tensor[:, :num_frames:skip_frames]
run_angles = run_angle_tensor[:, :num_frames:skip_frames]
wr_angles = torch.cat((walk_angles, run_angles))

animation_shape = wr_angles.shape[-2:]
animation_shape

In [ ]:
# use only walking angles
std, mean = torch.std_mean(walk_angles, dim=0)
std, mean = 1, 0
wr_angles_norm = (wr_angles - mean) / std
run_angles_norm = (run_angles - mean) / std
walk_angles_norm = (walk_angles - mean) / std

In [ ]:
# reshape
flatten = False
add_channel = True
make_frames = False
orig_shape_walk = walk_angles.shape
orig_shape_run = run_angles.shape
orig_shape_wr = wr_angles.shape

if add_channel:
    walk_angles_nr = torch.unsqueeze(walk_angles_norm, 1)
    run_angles_nr = torch.unsqueeze(walk_angles_norm, 1)
    wr_angles_nr = torch.unsqueeze(wr_angles_norm, 1)
elif flatten:
    walk_angles_nr = walk_angles_norm.reshape(
        orig_shape_walk[0], orig_shape_walk[1] * orig_shape_walk[2]
    )
    run_angles_nr = run_angles_norm.reshape(
        orig_shape_run[0], orig_shape_run[1] * orig_shape_run[2]
    )
    wr_angles_nr = wr_angles_norm.reshape(
        orig_shape_wr[0], orig_shape_wr[1] * orig_shape_wr[2]
    )
elif make_frames:
    walk_angles_nr = walk_angles_norm.reshape(
        orig_shape_walk[0] * orig_shape_walk[1], orig_shape_walk[2]
    )
    run_angles_nr = run_angles_norm.reshape(
        orig_shape_run[0] * orig_shape_run[1], orig_shape_run[2]
    )
    wr_angles_nr = wr_angles_norm.reshape(
        orig_shape_wr[0] * orig_shape_wr[1], orig_shape_wr[2]
    )
wr_angles_nr.shape

In [ ]:
# walk_angles_rev  = torch.swapaxes(walk_angles, 2, 1)
data = torch.utils.data.TensorDataset(wr_angles_nr)
data_walk = torch.utils.data.TensorDataset(walk_angles_nr)
data_run = torch.utils.data.TensorDataset(run_angles_nr)

In [ ]:
#######
DIR = "../figures/interpolate_shape/"
SET_NAME = "walk_residual_5"
PATH_FIGURES = os.path.join(DIR, SET_NAME)
if not os.path.exists(PATH_FIGURES):
    os.makedirs(PATH_FIGURES)
########

event_shape = data[0][0].shape
base_dist = dist.Independent(
    dist.Normal(loc=torch.zeros(event_shape), scale=torch.ones(event_shape)), 3
)
lr_scheduler = lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, mode="min", factor=0.5, patience=10, verbose=True
)
# def get_flow(ji)
MODEL_PARAMS = {
    "model": [sf.nf.get_flow],
    "get_transform": [sf.transforms.get_residual_transform],
    "base_dist": [base_dist],
    "inverse_model": [False],
    "compose": [True],
}
num_layers = 5
MODEL_PARAMS_EXPERIMENT = {
    # "num_blocks_per_layer": [2],
    "CNN": [[True] * num_layers] * 2,
    "hidden_features": [[7] * num_layers] * 2,
    "hidden_layers": [[2] * num_layers] * 2,
    "n_exact_terms": [[6] * num_layers],
    "n_samples": [[20] * num_layers],
}

TRAINING_PARAMS = {
    "batch_size": [10],
    "compute_loss": [sf.nf.monte_carlo_dkl_loss],
    "post_batch": [sf.get_post_step_lipchitz(5)],
    "verbose": True,
}
# extend the previous dict with the zip of this
TRAINING_PARAMS_EXPERIMENT = {
    "verbose_interval": [10],
    "optimizer": ["ADAM"],
    "num_epochs": [300],
    "learning_rate": [0.1],
    "lr_scheduler": [lr_scheduler],
}

In [ ]:
# create iterators
model_params_iter_1 = etorch.create_subdictionary_iterator(MODEL_PARAMS)
# model_params_iter = chain.from_iterable((model_params_iter_1, model_params_iter_2))

model_exp_iter = etorch.create_subdictionary_iterator(
    MODEL_PARAMS_EXPERIMENT, product=False
)
exp_model_params_iter = etorch.add_dictionary_iterators(
    model_exp_iter, model_params_iter_1
)

training_params_iter = etorch.create_subdictionary_iterator(TRAINING_PARAMS)
training_exp_iter = etorch.create_subdictionary_iterator(
    TRAINING_PARAMS_EXPERIMENT, product=True
)
exp_training_params_iter = etorch.add_dictionary_iterators(
    training_exp_iter, training_params_iter
)

In [ ]:
cv_results = etorch.k_fold_cv_grid(
    model_params=exp_model_params_iter,
    fit=etorch.fit_module,
    training_params=exp_training_params_iter,
    data=data_walk,
)

In [ ]:
etorch.plotting.plot_result(
    path_figures=PATH_FIGURES,
    **cv_results,
)

In [ ]:
flow = cv_results["models"][0]

In [ ]:
for i in range(len(flow.bijector.bijectors)):
    flow.bijector.bijectors[i].model.iresblock.brute_force = True

In [ ]:
# flow = torch.load("flow_full_motion_2.pt")
# flow.eval()

In [ ]:
noise = base_dist.sample([100])

print("Log plots:")

print("Noise :", flow.log_prob(noise).mean().item())
print("Run data:", flow.log_prob(data_run[:][0]).mean().item())
# print("Train data:", flow.log_prob(data[0:20][0]).mean().item())
print("Walk data:", flow.log_prob(data_walk[:][0]).mean().item())

In [ ]:
i, j = 0, 8
num_int_points = 20
x_1 = data_walk[i : i + 1][0]
x_2 = data_walk[j : j + 1][0]
line_ = torch.linspace(0, 1, num_int_points)
line = torch.reshape(line_, (num_int_points, 1, 1, 1))
z_1 = flow.normalize(x_1)
z_2 = flow.normalize(x_2)
z_interp = z_1 * line + z_2 * (line - 1)

x_lin_interp = x_1 * line + x_2 * (line - 1)
x_lat_interp = flow.bijector.forward(z_interp)

In [ ]:
with torch.no_grad():
    lat_log_prob = torch.mean(
        torch.stack([flow.log_prob(x_lat_interp) for a in range(10)]), dim=0
    )
    lin_log_prob = torch.mean(
        torch.stack([flow.log_prob(x_lin_interp) for a in range(10)]), dim=0
    )
    plt.plot(line_, lat_log_prob, "-", label="Latent space interp.")
    plt.plot(line_, lin_log_prob, "-.", label="Feature space interp.")
    plt.xlabel("$t$")
    plt.ylabel("$\log p_{T(Z)}$")
plt.legend()
plt.savefig(
    os.path.join(
        PATH_FIGURES,
        "interpolation_log_prob.pdf",
    ),
    bbox_inches="tight",
    pad_inches=0,
)
plt.show()

In [ ]:
data_input_temp = data_walk[0:1][0]  # x_lin_interp*std + mean

s = data_input_temp.shape
frames = np.prod(s[:3])
data_input = (data_input_temp).reshape((frames,) + (s[3],))
test_anim = copy.deepcopy(walk_animations[0])
test_anim.from_numpy_array(sf.utils.data_to_motion_array(data_input))
skel = copy.deepcopy(run_skeletons[0])
anim = mayavi_animate(
    skel,
    test_anim,
    offset=[0, 0, 0],
    continuous=False,
    fixed_cam=False,
    frame_limit=-1,
    save_path="test.svg",
)

In [ ]:
data_input_temp = x_lat_interp * std + mean

s = data_input_temp.shape
frames = np.prod(s[:3])
data_input = (data_input_temp).reshape((frames,) + (s[3],))

test_sample = copy.deepcopy(walk_animations[1])
test_sample.from_numpy_array(sf.utils.data_to_motion_array(data_input))
skel = copy.deepcopy(run_skeletons[0])
anim = mayavi_animate(
    skel,
    test_sample,
    offset=[1, 1, 1],
    continuous=False,
    fixed_cam=False,
    frame_limit=-1,
    save_path="test.svg",
)

In [ ]:
np.linspace(0, 1, 3).reshape((3, 1))  # *np.array([0,1,2])

In [ ]:
x_lin_interp[0:1] - x_1

In [ ]:
x_1.shape

In [ ]:
line

In [ ]:
line_

In [ ]:
line